# 11/14/19

Okay, so today is the day that I start wrangling the medical dataset. First thing is first: get the catheter model working as a slicing function.

In [10]:
%load_ext autoreload
%autoreload 2

import os
os.chdir('/lfs/1/gangus/repositories/pytorch-classification')

import os.path as osp
import json
import pickle

import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
# import matplotlib.font_manager as font_manager

# import models.cifar as models
# from analysis.analysis_utils import load_trained_model, fetch_dataloaders, get_coarse_accuracies

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
{
    'dataroot': True,
    'batchSize': 1,
    'loadSize': 512,
    
}     
dataroot', required=True, help='path to images (should have subfolders trainA, trainB, valA, valB, etc)')
batchSize', type=int, default=1, help='input batch size')
loadSize', type=int, default=512, help='scale images to this size')
fineSize', type=int, default=512, help='then crop to this size')
input_nc', type=int, default=3, help='# of input image channels')
output_nc', type=int, default=3, help='# of output image channels')
ngf', type=int, default=64, help='# of gen filters in first conv layer')
which_model_netG', type=str, default='srcnn', help='selects model to use for netG')
gpu_ids', type=str, default='0', help='gpu ids: e.g. 0  0,1,2, 0,2. use -1 for CPU')
name', type=str, default='experiment_name', help='name of the experiment. It decides where to store samples and models')
dataset_mode', type=str, default='alignedsrcnn', help='chooses how datasets are loaded.')
model', type=str, default='srcnn',
                     help='chooses which model to use')
which_direction', type=str, default='AtoB', help='AtoB or BtoA')
nThreads', default=2, type=int, help='# threads for loading data')
checkpoints_dir', type=str, default='./checkpoints', help='models are saved here')
norm', type=str, default='instance', help='instance normalization or batch normalization')
serial_batches', action='store_true', help='if true, takes images in order to make batches, otherwise takes them randomly')
display_winsize', type=int, default=256,  help='display window size')
display_id', type=int, default=1, help='window id of the web display')
display_port', type=int, default=8097, help='visdom port of the web display')
no_dropout', action='store_true', help='no dropout for the generator')
max_dataset_size', type=int, default=float("inf"), help='Maximum number of samples allowed per dataset. If the dataset directory contains more than max_dataset_size, only a subset is loaded.')
resize_or_crop', type=str, default='none', help='scaling and cropping of images at load time [resize_and_crop|crop|scale_width|scale_width_and_crop]')
no_flip', action='store_true', help='if specified, do not flip the images for data augmentation')
init_type', type=str, default='normal', help='network initialization [normal|xavier|kaiming|orthogonal]')

self.parser.add_argument('--ntest', type=int, default=float("inf"), help='# of test examples.')
        self.parser.add_argument('--results_dir', type=str, default='./results/', help='saves results here.')
        self.parser.add_argument('--aspect_ratio', type=float, default=1.0, help='aspect ratio of result images')
        self.parser.add_argument('--phase', type=str, default='test', help='train, val, test, etc')
        self.parser.add_argument('--which_epoch', type=str, default='latest', help='which epoch to load? set to latest to use latest cached model')
        self.parser.add_argument('--how_many', type=int, default=100, help='how many test images to run')
        self.parser.add_argument('--sourceoftest', type=str, default='internal', help='internal or external')

        self.isTrain = False

In [7]:
config = experiment_configs['baseline']

superclass = config['superclass']
cifar_type = config['cifar_type']
model_name = config['model_name']
model_args = config['model_args']
checkpoint_dir = config['checkpoint_dir']
trained_model = load_trained_model(cifar_type, model_name, model_args, checkpoint_dir)

data_dir = config['data_dir']
dataset_configs = config['dataset_configs']
dataloader_configs = config['dataloader_configs']
dataloaders = fetch_dataloaders(data_dir, cifar_type, superclass, dataset_configs, dataloader_configs)

# Getting class-to-index maps for CIFAR100
class_to_idx = {}
class_to_idx['super'] = dataloaders['test'].dataset.class_to_idx
class_to_idx['sub'] = dataloaders['test'].dataset.fine_class_to_idx

idx_to_class = {}
for tp in ['super', 'sub']:
    idx_to_class[tp] = {v:k for k, v in class_to_idx[tp].items()}
    
trained_model = trained_model.to('cpu')
trained_model = trained_model.module.to('cpu')

Using 20 classes...
Using coarse labels...
Using coarse labels...


In [9]:
filenames_all = []
coarse_labels_all = []
fine_labels_all = []
fine_preds_all = []
preds_all = []
losses_all = []
softmax_all = []
features_all = []

t = tqdm(total=len(dataloaders['test']))
for batch_idx, (inputs, 
                targets, 
                coarse_labels, 
                fine_labels, 
                filenames) in enumerate(dataloaders['test']):
    
    inputs, targets = inputs.cpu(), targets.cpu()
    inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)
    
    outputs = trained_model(inputs)

    _, preds = outputs.topk(1, 1, True, True)
    soft = torch.nn.Softmax(dim=-1)(outputs)
    # feats = extract_resnext_features(trained_model,inputs)
    
    coarse_labels_all+=list([int(a) for a in coarse_labels])
    fine_labels_all+=list([int(a) for a in fine_labels])
    preds_all+=list([int(a) for a in preds])
    softmax_all+=list([a.detach().cpu().numpy() for a in soft])
    # features_all+=list([a.detach().cpu().numpy() for a in feats])
    filenames_all+=list(filenames)
    t.update()
t.close()


100%|██████████| 100/100 [01:52<00:00,  1.13s/it]


In [ ]:
preds_dict = {
    'filename':filenames_all,
    'fine_labels':fine_labels_all,
    'coarse_labels':coarse_labels_all,
    'preds':preds_all,
    'fine_labels_string': [idx_to_class['sub'][a] for a in fine_labels_all],
    'coarse_labels_string': [idx_to_class['super'][a] for a in coarse_labels_all],
    # 'features':features_all,
    'softmax':softmax_all,
}
with open(f"{checkpoint_dir}/predictions.pkl",'wb') as f:
   pickle.dump(preds_dict, f)

prediction_df = pd.DataFrame(preds_dict)

In [ ]:
coarse_results = get_coarse_accuracies(prediction_df)

In [ ]:
# Melting dataframe for plotting
plot_df = pd.DataFrame(coarse_results).reset_index()
plot_df = plot_df.rename(columns={"index": "Superclass"})
plot_df = plot_df.melt(id_vars=['Superclass'], var_name='Subset', value_name='Accuracy')

In [ ]:
sns.set(font_scale=1.2) 
sns.set_style("whitegrid")
sns.set_style({'font.family':'serif'})

font = font_manager.FontProperties(family='Dejavu Serif',
                                   style='normal', size=16)
plt.rcParams.update({'font.family': 'serif'})

palette = [sns.xkcd_rgb["black"]] + 5*[sns.xkcd_rgb["blue"]]
sns.set(font_scale=1.5)
f = plt.figure(figsize=(10,10))
ax = sns.stripplot(x="Superclass", y="Accuracy", hue="Subset", data=plot_df, jitter=False, s=20, palette=palette, alpha=0.5)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90,fontdict={'family':'Dejavu Serif'})
# ax.set_yticklabels(ax.get_yticklabels(),fontdict={'family':'Dejavu Serif'})
ax.set_xlabel(ax.get_xlabel(),fontdict={'family':'Dejavu Serif'})
ax.set_ylabel(ax.get_ylabel(),fontdict={'family':'Dejavu Serif'})
h,l = ax.get_legend_handles_labels()
l[1] = 'Subclasses'
l[0] = 'Superclass'
ax.legend_.remove()
ax.legend(h[0:2],l[0:2], ncol=1, loc='lower right',prop=font)
ax.set_ylim([-0.02,1.02])
plt.tight_layout()
plt.savefig(f'{checkpoint_dir}/Superclass-Subclass-CIFAR-100-Correct-Val-v3.png',format='png')
plt.show()